This is a forked version of another person's public kernel.  
The original author had lost his account.  
In the interest of fairness, I'm making it public.  

Deleted Topic  
[https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/256725]

In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 8)

In [ ]:
INPUT_PATH = '../input/optiver-realized-volatility-prediction/'

In [ ]:
train = pd.read_csv(INPUT_PATH + 'train.csv')

In [ ]:
def mflip(x):
    return np.where(x > 0.5, x - 1, x)

In [ ]:
all_prices = {}

In [ ]:
%%time
for sid in train.stock_id.unique():
#     if sid >= 31: continue;
    print(sid)
    book = pd.read_parquet(INPUT_PATH + '/book_train.parquet/stock_id={}'.format(sid))
    prices = {}
    for tid in book.time_id.unique():
        bt = book[book.time_id == tid]
        chgs = np.hstack(    [np.log(bt[col]).diff().fillna(0).unique()
                for col in [b for b in book.columns if 'price' in b]] )        
        chgs = chgs[chgs != 0]
        if len(chgs) < 6: continue;

        minchg = np.abs(chgs).min()
        mults = np.abs(chgs)/ minchg
    #     plt.hist(mflip(mults %1 ), bins =250); plt.figure()
    #     plt.plot( ( mflip(mults % 1 ) / mults) )

        if np.max( mflip(mults % 1 ) / mults) > 0.2:
            print('*x*'); continue;

        rem = ( mflip(mults % 1 ) / mults).mean() * minchg
        rem

        minchg += rem

        if random.random() < 1/500 and len(chgs) > 10: print(1/ minchg)
    #     break;
        prices[tid] = 1/minchg
    
    all_prices[sid] = (prices, len(chgs))
    print()

In [ ]:
import pickle

In [ ]:
pickle.dump(all_prices, open('all_prices.pkl', 'wb'))